## 📝 보쉬 생산 라인 불량 예측 따라해보기

안녕하세요! 이 노트북은 '보쉬'라는 회사에서 어떤 부품이 불량일지 예측하는 모델을 만들어보는 과정을 담고 있어요. 데이터 분석이 처음인 중학생 친구들도 쉽게 따라 할 수 있도록 차근차근 설명할게요. 우리가 함께할 과정은 다음과 같아요.

1.  **데이터 불러오기**: 어떤 부품이 불량인지 아닌지 정보가 담긴 데이터를 컴퓨터로 가져올 거예요.
2.  **데이터 탐험하기**: 가져온 데이터가 어떻게 생겼는지, 어떤 정보가 들어있는지 살펴볼 거예요.
3.  **모델 훈련시키기**: 인공지능 모델에게 어떤 부품이 불량인지 학습시킬 거예요.
4.  **모델 평가하기**: 우리가 만든 모델이 얼마나 예측을 잘하는지 확인해볼 거예요.

**✅ 준비물**: 이 코드를 실행하기 전에 [캐글 대회 페이지](https://www.kaggle.com/competitions/bosch-production-line-performance/data)에 가서 `train_numeric.csv` 파일을 다운로드하고, 이 노트북 파일과 같은 폴더에 넣어주세요!

### 1단계: 데이터 분석에 필요한 도구들 불러오기

In [1]:
# pandas는 엑셀 시트처럼 생긴 데이터를 다룰 때 사용하는 아주 유용한 도구예요.
# 데이터를 불러오고, 자르고, 붙이는 등 다양한 작업을 할 수 있게 도와줘요.
import pandas as pd


# scikit-learn은 인공지능 모델을 만들 때 필요한 도구들이 모여있는 곳이에요.
# 여기에서 'DecisionTreeClassifier'라는 모델과 'train_test_split'이라는 데이터 나누기 도구를 가져올 거예요.
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# matplotlib와 seaborn은 데이터를 그래프로 예쁘게 보여주는 도구예요.
import matplotlib.pyplot as plt
import seaborn as sns

### 2단계: 데이터 불러오기

이제 보쉬의 부품 데이터가 담긴 `train_numeric.csv` 파일을 불러올 거예요. 이 파일은 크기가 매우 크기 때문에, 한 번에 모든 데이터를 불러오면 컴퓨터가 힘들어할 수 있어요. 그래서 `nrows`를 이용해 앞부분의 10,000개 부품 데이터만 샘플로 가져와서 분석해볼게요.

In [2]:
# 파일 경로를 변수에 저장해요. 나중에 다시 쓰기 편하게 하려고요.
file_path = 'train_numeric.csv'

try:
    # pd.read_csv를 사용해서 CSV 파일을 읽어올 거예요.
    # nrows=10000은 파일의 첫 10,000줄만 읽어오라는 뜻이에요. 데이터가 너무 커서 맛보기로 조금만 보는 거죠!
    data = pd.read_csv(file_path, nrows=10000)
    
    # 데이터를 잘 불러왔는지 확인하기 위해, 맨 앞 5줄을 화면에 출력해볼게요.
    # .head()는 데이터의 첫 부분을 살짝 보여주는 명령어예요.
    print("성공적으로 데이터를 불러왔습니다! 데이터의 첫 5줄은 다음과 같습니다:")
    print(data.head())

except FileNotFoundError:
    # 만약 파일을 찾을 수 없다는 에러가 나면, 사용자에게 파일을 다운로드하라고 알려줘요.
    print(f"오류: '{file_path}' 파일을 찾을 수 없습니다.")
    print("캐글 사이트에서 데이터를 다운로드하여 이 노트북과 같은 폴더에 저장했는지 확인해주세요!")

오류: 'train_numeric.csv' 파일을 찾을 수 없습니다.
캐글 사이트에서 데이터를 다운로드하여 이 노트북과 같은 폴더에 저장했는지 확인해주세요!


### 3단계: 데이터 탐험하기 (EDA)

데이터를 불러왔으니, 이제 데이터가 어떻게 생겼는지 자세히 살펴볼 시간이에요. 이걸 '탐험적 데이터 분석(Exploratory Data Analysis, EDA)'이라고 불러요. 데이터와 친해지는 과정이죠!

In [ ]:
# 데이터의 크기를 확인해볼게요. (몇 개의 행, 몇 개의 열)로 보여줘요.
# 행(row)은 부품 하나하나를, 열(column)은 각 부품의 측정값이나 특징을 의미해요.
print(f"우리 데이터는 {data.shape[0]}개의 부품(행)과 {data.shape[1]}개의 특징(열)으로 이루어져 있어요.")

# 'Response' 열은 우리가 예측하고 싶은 목표예요. 0은 '정상', 1은 '불량'을 의미해요.
# 어떤 부품이 정상이고 불량인지 그 개수를 세어볼게요.
print("
정상(0)과 불량(1) 부품의 개수:")
print(data['Response'].value_counts())

# 정상과 불량 부품의 비율을 막대그래프로 그려볼게요.
plt.figure(figsize=(7, 5)) # 그래프의 크기를 정해요.
sns.countplot(x='Response', data=data) # 'Response' 열의 값을 기준으로 개수를 세어 막대그래프를 그려요.
plt.title('정상 vs 불량 부품 개수') # 그래프의 제목을 붙여요.
plt.show() # 그래프를 화면에 보여줘요.

그래프를 보면, 불량(1)인 부품보다 정상(0)인 부품이 훨씬 많은 것을 알 수 있어요. 이렇게 데이터가 한쪽으로 치우쳐져 있는 것을 '불균형 데이터'라고 불러요. 이런 데이터로 모델을 만들 때는 조금 더 신경 써야 할 부분들이 있답니다.

### 4단계: 모델을 훈련시키기 위한 데이터 준비

이제 인공지능 모델을 훈련시킬 시간이에요. 모델이 잘 학습하려면, 데이터를 '문제지'와 '정답지'로 나누어주어야 해요.

- **문제지 (X)**: 부품의 여러 측정값들 (특징, Features)
- **정답지 (y)**: 각 부품이 불량인지 아닌지 (목표, Target)

그리고 데이터에 비어있는 값(결측치, Missing Values)이 있으면 모델이 헷갈릴 수 있으니, 비어있는 칸은 0으로 채워줄게요.

In [ ]:
# 'Id' 열은 부품의 고유 번호라서 예측에 필요 없고, 'Response' 열은 정답이니까 문제지에서 빼줄게요.
# X 변수에는 문제지(특징)를 담아요.
X = data.drop(['Id', 'Response'], axis=1)

# y 변수에는 정답지(목표)를 담아요.
y = data['Response']

# 데이터에 비어있는 값(NaN)이 있으면 모델이 학습할 수 없어요.
# .fillna(0)는 비어있는 모든 칸을 0으로 채워주는 명령어예요.
X = X.fillna(0)

# 이제 문제지와 정답지가 잘 준비되었는지 확인해볼까요?
print("문제지 (X)의 첫 5줄:")
print(X.head())
print("
정답지 (y)의 첫 5줄:")
print(y.head())

### 5단계: 모델 훈련시키기

이제 진짜로 모델을 훈련시켜볼게요. '결정 트리(Decision Tree)'라는 모델을 사용할 거예요. 이 모델은 '만약 A 특징이 10보다 크면, 불량일 확률이 높다'와 같이 '예/아니오' 질문을 반복하면서 예측하는 방식이라 이해하기 쉬워요.

가지고 있는 데이터 중 80%는 모델을 훈련시키는 데 사용하고(훈련 데이터), 나머지 20%는 모델이 얼마나 잘 배웠는지 시험해보는 데 사용할 거예요(테스트 데이터).

In [ ]:
# train_test_split 함수는 데이터를 훈련용과 테스트용으로 나눠줘요.
# test_size=0.2는 전체 데이터의 20%를 테스트용으로 쓰겠다는 의미예요.
# random_state=42는 데이터를 나눌 때 항상 똑같은 방식으로 나누게 해서, 실행할 때마다 결과가 바뀌지 않도록 해요.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# DecisionTreeClassifier 모델을 만들어요. max_depth=5는 트리의 깊이를 5단계로 제한해서 너무 복잡해지지 않게 해요.
model = DecisionTreeClassifier(max_depth=5, random_state=42)

# .fit()은 모델을 훈련시키는 명령어예요. 훈련용 문제지(X_train)와 훈련용 정답지(y_train)를 주고 학습시켜요.
model.fit(X_train, y_train)

print("모델 훈련이 완료되었습니다!")

### 6단계: 모델 평가하기

훈련이 끝났어요! 이제 우리가 만든 모델이 얼마나 똑똑한지 확인해볼 시간이에요. 한 번도 본 적 없는 테스트 데이터를 주고, 얼마나 정답을 잘 맞히는지 평가해볼게요.

In [ ]:
# .predict()는 훈련된 모델을 사용해서 예측을 하는 명령어예요.
# 테스트용 문제지(X_test)를 주고 정답을 예측해보라고 시켜요.
predictions = model.predict(X_test)

# accuracy_score는 모델의 예측(predictions)과 실제 정답(y_test)을 비교해서 정확도를 계산해줘요.
# 정확도는 전체 문제 중에서 모델이 맞힌 문제의 비율을 의미해요.
accuracy = accuracy_score(y_test, predictions)

print(f"우리가 만든 모델의 정확도는 {accuracy * 100:.2f}% 입니다!")

### 마무리

축하합니다! 보쉬 데이터를 이용해서 불량을 예측하는 인공지능 모델을 직접 만들어봤어요. 물론 실제 대회에서는 더 복잡하고 어려운 방법들을 사용해야 높은 점수를 받을 수 있지만, 오늘 우리는 데이터 분석의 가장 기본적인 과정을 체험해봤어요.

데이터를 불러오고, 살펴보고, 모델을 만들어 평가하는 이 과정을 잘 기억해두면 앞으로 더 신기하고 재미있는 데이터 분석 프로젝트들을 해볼 수 있을 거예요. 수고 많으셨습니다!